# SQLAlchemy and Alembic


### 최영선
### yeongseon.choe@gmail.com

"If I have seen future. it is by importing from the code of giants"
- Definitely Not isaac Newton

How to use database in Python


Installing a Sqlalchemy

In [8]:
import sqlalchemy

print( sqlalchemy.__version__)

1.1.11


In [9]:
print("test")

test
